# Toggle Automatic Switching: Is it possible?

*(July 4, 2025 - by Stefan Behrens)*

This will only be interesting for people who are running home automation devices using an AVM FRITZ!Box router. Specifically, this concerns smart plugs from the AVM FRITZ! product line. These can operate as timer switches with user specified on and off periods (specified per week day). I'm using the timer switch feature for my espresso machine, which needs to heat up for ~15 minutes before use, to have it ready when I need it (especially in the morning). Since I regularly commute on weekends, I have to toggle timer switching off when I leave and back on when I return home. 

Unfortunately, at this point (July 4, 2025) neither the official FRITZ! Smart Home app (for Android) nor the `fritzconnection` API for Python offern an easy way to change these settings. I have to log into the FRITZ!Box web interface every single time, browse through various menus and tabs, scroll down to find a check box to toggle the timer switch, click it, and then click 'Confirm Changes'. This is rather tedious and I would like to have an easier  solution, ideally a one-click solution.

When I first came across `fritzconnection`, I was initially hoping to find a way to make these changes. But I soon realized that it's currently (again, July 4, 2025) only possible to *get* the current timer switch state ('automatic' or 'manual'), not to *set* is. Ever since then, I've been curious to find a way to do this myself.

At some point, I dared to take a look under the hood of `fritzconnection` but I soon realized that my network communication skills were not up to the task, yet. Nevertheless, I'm convinced that there's a way to do this. After all, the timer information must leave my browser in one way or another to find its way to the FRITZ!Box. If only I knew how this worked already... Since I don't, I'll have to figure it out. On the plus side, this should give me the chance to improve my network communication skills. So let's go!

## Google Chrome to the rescue

At first I thought that I would need to use something like Wireshark to track down the packets containing the information sent from my browser to the FRITZ!Box. But then I realized that said browser, Google Chrome, might be enough. In fact, if you're also using Chrome and have never pressed F12 before (or clicked 'Inspect' after right clicking something), you're in for a treat. This makes the **Chrome DevTools Console** appear which contains all kinds of information about whatever is going on in open tabs. 

In particular, the **Network** tab in the DevTools console provides a log of pretty much everything that Chrome receives or sends. Finding the messages for toggling the timer switch setting was suprisingly easy. Unfortunately, making sense of it still isn't. Here's what the messages look like (copied using *'Copy as fetch'*): 

```python
# Example: Toggling the automatic switching option _on_
fetch("http://fritz.box/data.lua", {
  "headers": {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,de;q=0.8,nl;q=0.7,fr;q=0.6",
    "content-type": "application/x-www-form-urlencoded"
  },
  "referrer": "http://fritz.box/",
  "body": "xhr=1&sid=249cfb15ebf855ce&device=16&view=&back_to_page=%2Fsmarthome%2Fdevices.lua&switchtimer=weekly&graphState=1&timer_item_0=0730%3B1%3B31&timer_item_1=1100%3B0%3B31&timer_item_2=1300%3B1%3B31&timer_item_3=1430%3B0%3B31&timer_item_4=1600%3B1%3B31&timer_item_5=1700%3B0%3B31&timer_item_6=0800%3B1%3B96&timer_item_7=1145%3B0%3B96&timer_item_8=1315%3B1%3B96&timer_item_9=1645%3B0%3B96&zufall_from_day=5&zufall_from_month=7&zufall_from_year=2025&zufall_to_day=11&zufall_to_month=7&zufall_to_year=2025&zufall_from_hh=8&zufall_from_mm=00&zufall_to_hh=18&zufall_to_mm=00&zufall_duration=60&countdown_onoff=0&countdown_off_hh=0&countdown_off_mm=0&rythmisch_on=0&rythmisch_off=0&single_onoff=1&single_day=5&single_month=7&single_year=2025&single_hh=17&single_mm=15&single_duration=-1&latitude=52%2C52&longitude=13%2C34&sunset_on_relative=00%3A00&sunset_off_relative=00%3A00&sunset_off_duration=00%3A00&sunrise_on_relative=00%3A00&sunrise_off_relative=00%3A00&sunrise_off_duration=00%3A00&calendarname=&stand_by_power=&stand_by_duration=&apply=&lang=de&page=home_auto_timer_view",
  "method": "POST",
  "mode": "cors",
  "credentials": "omit"
});

# Example: Toggling the automatic switching option _off_
fetch("http://fritz.box/data.lua", {
  "headers": {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,de;q=0.8,nl;q=0.7,fr;q=0.6",
    "content-type": "application/x-www-form-urlencoded"
  },
  "referrer": "http://fritz.box/",
  "body": "xhr=1&sid=249cfb15ebf855ce&device=16&view=&back_to_page=%2Fsmarthome%2Fdevices.lua&switchautomatic=on&switchtimer=weekly&graphState=1&timer_item_0=0730%3B1%3B31&timer_item_1=1100%3B0%3B31&timer_item_2=1300%3B1%3B31&timer_item_3=1430%3B0%3B31&timer_item_4=1600%3B1%3B31&timer_item_5=1700%3B0%3B31&timer_item_6=0800%3B1%3B96&timer_item_7=1145%3B0%3B96&timer_item_8=1315%3B1%3B96&timer_item_9=1645%3B0%3B96&zufall_from_day=5&zufall_from_month=7&zufall_from_year=2025&zufall_to_day=11&zufall_to_month=7&zufall_to_year=2025&zufall_from_hh=8&zufall_from_mm=00&zufall_to_hh=18&zufall_to_mm=00&zufall_duration=60&countdown_onoff=0&countdown_off_hh=0&countdown_off_mm=0&rythmisch_on=0&rythmisch_off=0&single_onoff=1&single_day=5&single_month=7&single_year=2025&single_hh=17&single_mm=16&single_duration=-1&latitude=52%2C52&longitude=13%2C34&sunset_on_relative=00%3A00&sunset_off_relative=00%3A00&sunset_off_duration=00%3A00&sunrise_on_relative=00%3A00&sunrise_off_relative=00%3A00&sunrise_off_duration=00%3A00&calendarname=&stand_by_power=&stand_by_duration=&apply=&lang=de&page=home_auto_timer_view",
  "method": "POST",
  "mode": "cors",
  "credentials": "omit"
});
```

Good news: This looks Python compatible. Let's go ahead and extract the apparent dictionaries passed to the second argument of `fetch` and take a closer look at the long string in the `body` part which contains the interesting information.

In [22]:
toggle_off = {
  "headers": {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,de;q=0.8,nl;q=0.7,fr;q=0.6",
    "content-type": "application/x-www-form-urlencoded"
  },
  "referrer": "http://fritz.box/",
  "body": "xhr=1&sid=249cfb15ebf855ce&device=16&view=&back_to_page=%2Fsmarthome%2Fdevices.lua&switchtimer=weekly&graphState=1&timer_item_0=0730%3B1%3B31&timer_item_1=1100%3B0%3B31&timer_item_2=1300%3B1%3B31&timer_item_3=1430%3B0%3B31&timer_item_4=1600%3B1%3B31&timer_item_5=1700%3B0%3B31&timer_item_6=0800%3B1%3B96&timer_item_7=1145%3B0%3B96&timer_item_8=1315%3B1%3B96&timer_item_9=1645%3B0%3B96&zufall_from_day=5&zufall_from_month=7&zufall_from_year=2025&zufall_to_day=11&zufall_to_month=7&zufall_to_year=2025&zufall_from_hh=8&zufall_from_mm=00&zufall_to_hh=18&zufall_to_mm=00&zufall_duration=60&countdown_onoff=0&countdown_off_hh=0&countdown_off_mm=0&rythmisch_on=0&rythmisch_off=0&single_onoff=1&single_day=5&single_month=7&single_year=2025&single_hh=17&single_mm=15&single_duration=-1&latitude=52%2C52&longitude=13%2C34&sunset_on_relative=00%3A00&sunset_off_relative=00%3A00&sunset_off_duration=00%3A00&sunrise_on_relative=00%3A00&sunrise_off_relative=00%3A00&sunrise_off_duration=00%3A00&calendarname=&stand_by_power=&stand_by_duration=&apply=&lang=de&page=home_auto_timer_view",
  "method": "POST",
  "mode": "cors",
  "credentials": "omit"
}

toggle_on = {
  "headers": {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,de;q=0.8,nl;q=0.7,fr;q=0.6",
    "content-type": "application/x-www-form-urlencoded"
  },
  "referrer": "http://fritz.box/",
  "body": "xhr=1&sid=249cfb15ebf855ce&device=16&view=&back_to_page=%2Fsmarthome%2Fdevices.lua&switchautomatic=on&switchtimer=weekly&graphState=1&timer_item_0=0730%3B1%3B31&timer_item_1=1100%3B0%3B31&timer_item_2=1300%3B1%3B31&timer_item_3=1430%3B0%3B31&timer_item_4=1600%3B1%3B31&timer_item_5=1700%3B0%3B31&timer_item_6=0800%3B1%3B96&timer_item_7=1145%3B0%3B96&timer_item_8=1315%3B1%3B96&timer_item_9=1645%3B0%3B96&zufall_from_day=5&zufall_from_month=7&zufall_from_year=2025&zufall_to_day=11&zufall_to_month=7&zufall_to_year=2025&zufall_from_hh=8&zufall_from_mm=00&zufall_to_hh=18&zufall_to_mm=00&zufall_duration=60&countdown_onoff=0&countdown_off_hh=0&countdown_off_mm=0&rythmisch_on=0&rythmisch_off=0&single_onoff=1&single_day=5&single_month=7&single_year=2025&single_hh=17&single_mm=16&single_duration=-1&latitude=52%2C52&longitude=13%2C34&sunset_on_relative=00%3A00&sunset_off_relative=00%3A00&sunset_off_duration=00%3A00&sunrise_on_relative=00%3A00&sunrise_off_relative=00%3A00&sunrise_off_duration=00%3A00&calendarname=&stand_by_power=&stand_by_duration=&apply=&lang=de&page=home_auto_timer_view",
  "method": "POST",
  "mode": "cors",
  "credentials": "omit"
}

# extract 'body' string and split at '&'
toggle_off_settings = toggle_off["body"].split("&")
toggle_on_settings = toggle_on["body"].split("&")
# print info about toggle_on_settings
print("Lenght of toggle_off_settings:", len(toggle_off_settings)) # --> 55
print("Lenght of toggle_on_settings: ", len(toggle_on_settings)) # --> 54
for info in toggle_on_settings:
    print([info])

Lenght of toggle_off_settings: 54
Lenght of toggle_on_settings:  55
['xhr=1']
['sid=249cfb15ebf855ce']
['device=16']
['view=']
['back_to_page=%2Fsmarthome%2Fdevices.lua']
['switchautomatic=on']
['switchtimer=weekly']
['graphState=1']
['timer_item_0=0730%3B1%3B31']
['timer_item_1=1100%3B0%3B31']
['timer_item_2=1300%3B1%3B31']
['timer_item_3=1430%3B0%3B31']
['timer_item_4=1600%3B1%3B31']
['timer_item_5=1700%3B0%3B31']
['timer_item_6=0800%3B1%3B96']
['timer_item_7=1145%3B0%3B96']
['timer_item_8=1315%3B1%3B96']
['timer_item_9=1645%3B0%3B96']
['zufall_from_day=5']
['zufall_from_month=7']
['zufall_from_year=2025']
['zufall_to_day=11']
['zufall_to_month=7']
['zufall_to_year=2025']
['zufall_from_hh=8']
['zufall_from_mm=00']
['zufall_to_hh=18']
['zufall_to_mm=00']
['zufall_duration=60']
['countdown_onoff=0']
['countdown_off_hh=0']
['countdown_off_mm=0']
['rythmisch_on=0']
['rythmisch_off=0']
['single_onoff=1']
['single_day=5']
['single_month=7']
['single_year=2025']
['single_hh=17']
['single_mm

What to take away from this? Well, there are a couple of surprises:
* The messages contain not one but **over 50** settings! This might be the reason why it is not that easy to change the automatic switching state individually.
* I'm not sure if I've seen this many options in the web interface. I recognize some, but not all. 
* As a side note, the message contains an amusing mix of English and German. 

Unfortunately, I don't recognize the automatic switching state toggle. Since I'm only changing one thing, I'm expecting that both messages (on and off) share most entries. Let'S try to find the differeces:

In [23]:
# entries in toggle_on_settings but not in toggle_off_settings:
[entry for entry in toggle_on_settings if not entry in toggle_off_settings]

['switchautomatic=on', 'single_mm=16']

In [24]:
# entries in toggle_off_settings but not in toggle_on_settings:
[entry for entry in toggle_off_settings if not entry in toggle_on_settings]

['single_mm=15']

More surprises and some confusion:
* The meaning of `'switchautomatic=on'` is clear. But why is there no `'switchautomatic=off'`?
* And what the hell is `single_mm` supposed to do?

Well, I don't know about you, but I've seen enough for today. 

### Summary and conclusions:

* The Chrome DevTools Console is even more useful than I thought.
* The automatic switching state cannot be changed individually. Over 50 other settings are involved.
* If I want to change only the automatic switching state, I first have to retrieve *all* involved settings. 
* I *might* be able to add this feature to `fritzconnection`, but I'll have to put in a good amount of work.
* I might also try to get in touch with the developer of `fritzconnection`.

*(to be continued...)*
